# Generation of DataFrames for Comparing Modalities

This notebook corresponds to the section titled **"Generation of DataFrames to Compare EEG and MEG Modalities with fMRI"** and utilizes the `generation_comparison_modalities_df` function from the `MEEG_fMRI_whole_compa_script.py` script.

In this notebook, you will compute the distances between the Peak of Activation (POA) of EEG (or MEG) and fMRI activation maps for each condition and each time point of EEG (or MEG). Similarly, distances will be calculated for the weighted Center of Gravity (wCOG). Additionally, the overlap between the peak clusters of EEG (or MEG) and the closest fMRI clusters will be analyzed.

All results will be saved into DataFrames for further analysis.

## Import the necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import nibabel as nib
from pathlib import Path
import time
import sys

# Personal Imports
# Add the directory that contains the utils package to sys.path
sys.path.append(str(Path('..').resolve()))
from utils.generation_compa_df_utils import *
from utils.df_utils import *
from utils.utils import save_df_to_csv, found_condition_file, setup_logging, trackprogress
from utils.mask_utils import masking_data_w_roi

## Define path ot data

Before running the notebooks, make sure to update the path in `config.py` to match your local setup.
- **`LOCAL_DIR`**: Set this to the directory where the BIDS-formatted data will be stored.

In [ ]:
from config import LOCAL_DIR
local_dir = LOCAL_DIR

## Define the susbjects of the study

**Subject Exclusions and Issues:**

- **Subjects 1 and 2**: Excluded from analysis because their results in the native space are not available.
- **Subject 11**: Removed due to issues encountered during data acquisitions.
- **Subject 10**: Not available.

In [ ]:
# Create a list of subjects from 'sub-03' to 'sub-17'
subjects = [f"sub-{i:02}" for i in range(3, 18) if i not in [11, 10]]

# Print the updated list to verify
print(subjects)

## Define tasks and conditions of interest

The following parameters must remain consistent with those used during the data preparation step. To ensure this, they are loaded from the `config.py` file:

- **tasks_conditions:** A dictionary mapping each task to its corresponding conditions.
- **thres_param:** A dictionary specifying the alpha threshold and minimum cluster size used for EEG and MEG data thresholding.
- **do_brain_masking:** A flag indicating whether MEG data were brain-masked during the preparation phase.
- **CorrectionParam:** Class with the parameters used for thresholding MRI data.
- **pos:** The resolution of the MEG source estimates.
- **eeg_stc_algo:** A str corresponding to the algorithm used to compute EEG source estimates.
- **fmri_comparison_clusters**: A list containing the indices of the fMRI clusters that will be compared with the EEG and MEG clusters. If multiple fMRI clusters are present, only the closest one is saved in the comparison dataframe. *This allows you to manually select the specific fMRI cluster to compare with the other modalities.* The index corresponds to the order of the clusters based on their peak activation.
- **roi_masking**: Specifies whether to apply ROI-based masking to the fMRI, EEG, and MEG statistical maps. When set to `True`, the statistical maps will be masked before comparison using the ROIs associated with the current task, as defined by the atlas specified in the `ATLAS` parameter.
- **atlas**: The atlas from which the ROIs are extracted.

In [ ]:
from config import TASKS_CONDITIONS, THRESHOLD_PARAMS, DO_BRAIN_MASKING, CorrectionParam, EEG_STC_ALGO, POS, ROI_MASKING, ATLAS
# For each task associate the contrast of interest
tasks_conditions = TASKS_CONDITIONS

# Define thresholding parameters
thres_param = THRESHOLD_PARAMS

# MEG brain masked
do_brain_masking = DO_BRAIN_MASKING

# The meg stc reoslution
pos = POS

# Algo to used to compute source estimates
eeg_stc_algo = EEG_STC_ALGO

# Index of the fmri clusters to which the meg and eeg clusters will be compared to 
fmri_comparison_clusters = [1, 2, 3]

# Variable to choose if the statmap are masked for comparison
roi_masking = False

# The atlas from which the ROIs are extracted.
atlas = ATLAS

## Compute distance

#### Summary of Analysis Process

For each condition and task per subject:

1. **Threshold Files Retrieval**:
   - Retrieve threshold files for fMRI, EEG, and MEG.
   - If `roi_masking` is `True`, the threshold files are masked using the ROI corresponding to the task and selected atlas.

2. **Cluster Table Extraction**:
   - **EEG and MEG**: 
     - Extract cluster tables.
     - Obtain POA locations (in mm) and cluster sizes (in mm³) for the first POA clusters and the clusters map.
   - **fMRI**:
     - Extract cluster tables.
     - Obtain POA locations (in mm) and cluster sizes (in mm³) for the FMRI_COMPARISON_CLUSTERS clusters and the clusters map.
   - Use the `extract_cluster_info` function from `post_analysis_utils.py` for extraction.

3. **Distance Computation**:
   - Compute the Euclidean distance between EEG or MEG POA and each of the FMRI_COMPARISON_CLUSTERS fMRI POAs.
   - Identify the closest fMRI POA and record its index.
   - Utilize the `calculate_distances` function from `post_analysis_utils.py` for computation.

4. **Center of Gravity Calculation**:
   - Compute the weighted center of gravity (WCOG) for each cluster.
   - Perform distance calculations as described above using the `found_wcog` function from `post_analysis_utils.py`.

5. **Cluster Size and Overlap Analysis**:
   - Compute the size of each cluster.
   - Calculate the Jaccard index of overlap between the closest fMRI cluster and the EEG/MEG cluster.
   - The Jaccard index is defined as:
    **Jaccard Index** = (fMRI Cluster ∩ EEG/MEG Cluster) / (fMRI Cluster ∪ EEG/MEG Cluster)
   - Use the `compute_overlap` function from `post_analysis_utils.py` for this analysis.

6. **Data Storage**:
   - Save results in a DataFrame for each subject.
   - Compile an aggregate DataFrame summarizing data across all subjects.
   - Manage these DataFrames using functions defined in `df_utils.py`.


**Note:**
- When applying `ROI_MASKING`, it was observed that activations were often not detected. Consider performing the masking before thresholding to potentially improve accuracy.
- There may also be issues with how the ROIs are defined, which could affect the results.

### Details about the DataFrames Columns:

The DataFrames contain the following columns (they are defined in `df_utils.py`):

- **subject_name**: Identifier for the subject.
- **task**: Identifier for the task.
- **condition**: Identifier for the condition.
- **tpindex**: Identifier for the time point.
- **fmridxEeg**: Index of the fMRI cluster compared with the EEG cluster.
- **fmridxMeg**: Index of the fMRI cluster compared with the MEG cluster.

#### POA and COG:
- **eeg**: Coordinates of the EEG Peak of Activation (POA) or Center of Gravity (COG).
- **eegfmri**: Coordinates of the fMRI POA or COG cluster compared to the EEG.
- **meg**: Coordinates of the MEG POA or COG.
- **megfmri**: Coordinates of the fMRI POA or COG cluster compared to the MEG.
- **dist_x**: Distance between the EEG/MEG cluster and the fMRI cluster in the x-direction.
- **dist_y**: Distance between the EEG/MEG cluster and the fMRI cluster in the y-direction.
- **dist_z**: Distance between the EEG/MEG cluster and the fMRI cluster in the z-direction.
- **dist_norm**: Euclidean distance between the EEG/MEG cluster and the fMRI cluster.

#### Overlap and Cluster Size:
- **eeg**: Size of the EEG cluster in mm³.
- **fmrieeg**: Size of the fMRI cluster compared to the EEG cluster in mm³.
- **meg**: Size of the MEG cluster in mm³.
- **fmrimeg**: Size of the fMRI cluster compared to the MEG cluster in mm³.
- **OverlapEeg**: Jaccard index for the overlap between the EEG and fMRI clusters, calculated as \((\text{EEG} \cap \text{fMRI}) / (\text{EEG} \cup \text{fMRI})\).
- **OverlapMeg**: Jaccard index for the overlap between the MEG and fMRI clusters, calculated as \((\text{MEG} \cap \text{fMRI}) / (\text{MEG} \cup \text{fMRI})\).

Therefore, each row contains data for both EEG and MEG measurements taken at the same time point and condition.


In [ ]:
# Setup logging
logger = setup_logging(local_dir, 'logger-generation_df_comparison')

# Def of a DataFrame that will store the results of all subjects together
df_all_sub_ = {'POA': all_subject_POA_df(), 'COG': all_subject_COG_df(), 'Overlap': all_subject_overlap_df()}

# Start the time count to track the remaining computation time
start_time = time.time()

# Total number of subjects for progress tracking
tot_perf = len(subjects)

# loop through subjects
for count_perf, sub in enumerate(subjects, start=0):

    # Define the path to the subject derivatives directory
    sub_derivatives_outdir = Path(local_dir) / 'derivatives' / sub

    logger.info(f'Analyzing subject {sub}.')
    print(f'Analyzing subject {sub}.')

    trackprogress(count_perf, tot_perf, start_time, logger)
            
    # Define a DataFrame for this subject
    df_sub_ = {'POA': subject_POA_df(), 'COG': subject_COG_df(), 'Overlap': subject_overlap_df()}

    # Loop through tasks
    for task in tasks_conditions.keys():
        logger.info(f"  Analyzing task {task}.")
        print(f"  Analyzing task {task}.")
        
        # Get the correction_param for this task
        corr_param = CorrectionParam(task)
        corr_method = corr_param.fMRI_CORRECTION_METHOD
        mri_alpha = corr_param.fMRI_ALPHA
        mri_cluster_thres = corr_param.fMRI_CLUSTER_THRES
        mri_twosided = corr_param.fMRI_TWOSIDED

        # Def path to the mri results
        mri_path = sub_derivatives_outdir / 'func' / f'task-{task}' / f'corr-{corr_method}'

        # # Def path to the eeg results
        eeg_path = sub_derivatives_outdir/ 'eeg' / 'stc_interpolated' / f'task-{task}'
        
        # Def path to the meg results
        meg_path = sub_derivatives_outdir / 'meg' / 'stc_interpolated' / f'task-{task}'

        # Loop through conditions
        for condi in tasks_conditions[task]:
            print(f"    Analyzing condition {condi}.")
            logger.info(f"    Analyzing condition {condi}.")

            # Define file patterns
            eeg_pattern = eeg_path / f"{sub}_task-{task}_condition-{condi}_desc-eeg-stcinterpol_tp-*_stat-{eeg_stc_algo}_statmap_resamp_masked_topercent-{thres_param['EEG']['alpha']}_cluster-{thres_param['EEG']['cluster_thresh']}.nii.gz"
            
            if do_brain_masking == False:
                meg_pattern = meg_path / f"{sub}_task-{task}_condition-{condi}_desc-meg-stcinterpol_tp-*_pos-{pos}_stat-sLORETA_statmap_resamp_topercent-{thres_param['MEG']['alpha']}_cluster-{thres_param['MEG']['cluster_thresh']}.nii.gz"
            else:
                meg_path / f"{sub}_task-{task}_condition-{condi}_desc-meg-stcinterpol__tp-*_pos-{pos}_stat-sLORETA_statmap_resampmasked_topercent-{thres_param['MEG']['alpha']}_cluster-{thres_param['MEG']['cluster_thresh']}.nii.gz"
            
            mri_file = mri_path / f"{sub}_task-{task}_contrast-{condi}_desc-stat-z_statmap_masked_corr-{corr_method}_alpha-{mri_alpha}_cluster-{mri_cluster_thres}_twosided-{mri_twosided}.nii.gz"
            
            # Find EEG and MEG files
            try:
                eeg_files = found_condition_file(eeg_pattern)
                meg_files = found_condition_file(meg_pattern)
            except Exception as e:
                # Handle any exceptions during files searching
                logger.error(f"      {e}")
                raise(e)

            if roi_masking:
                try:
                    mri_file = masking_data_w_roi(mri_file, sub_derivatives_outdir, sub, task, condi, atlas)
                    logger.info(f"      Masking by the {task} roi of {mri_file} done ({atlas}).")
                except Exception as e:
                    logger.error(f"      {e}.")
                    raise(e)
                
            # Get the POA coordinates and the cluster_maps of the MRI contrast
            try:
                mri_poa_coords, mri_clusters_size, mri_cluster_map = extract_cluster_info(mri_file, mri_cluster_thres, cluster_ids = fmri_comparison_clusters, n_clusters=len(fmri_comparison_clusters))
                mri_poa_coords = np.array([[np.nan, np.nan, np.nan]]) if len(mri_poa_coords) == 0 else mri_poa_coords
                logger.info(f"      MRI POA coordinates computed.")
            except Exception as e:
                # Handle any exceptions during fMRI clusters study
                logger.error(f"      {e}")
                raise(e)

            # Compute the COG for the chosen MRI clusters
            try:
                mri_cogs_coords = found_wcog(mri_cluster_map[0], mri_file, cog_index = fmri_comparison_clusters)
                logger.info(f"      MRI COG coordinates computed.")
            except Exception as e:
                # Handle any exceptions during COG computation
                logger.error(f"      {e}")
                mri_cogs_coords = np.array([[np.nan, np.nan, np.nan]]) # Handle as needed
            

            # Loop through EEG amd MEG files
            for tp, (eeg_file, meg_file) in enumerate(zip(eeg_files, meg_files)):
                logger.info(f"      Analyzing tp {tp}:")

                if roi_masking:
                    try:
                        eeg_file = masking_data_w_roi(eeg_file, sub_derivatives_outdir, sub, task, condi, atlas)
                        logger.info(f"      Masking by the {task} roi of {eeg_file} done ({atlas}).")
                    except Exception as e:
                        logger.error(f"      {e}.")
                        raise(e)
                    try:
                        meg_file = masking_data_w_roi(meg_file, sub_derivatives_outdir, sub, task, condi, atlas)
                        logger.info(f"      Masking by the {task} roi of {meg_file} done ({atlas}).")
                    except Exception as e:
                        logger.error(f"      {e}.")
                        raise(e)

                # Get the POA coordinates
                eeg_poa_coords, eeg_clusters_size, eeg_cluster_map = extract_cluster_info(eeg_file, thres_param['EEG']['cluster_thresh'], cluster_ids=[1], n_clusters=1)
                logger.info(f"        EEG POA coordinates computed.")
                meg_poa_coords, meg_clusters_size, meg_cluster_map = extract_cluster_info(meg_file, thres_param['MEG']['cluster_thresh'], cluster_ids=[1], n_clusters=1)
                logger.info(f"        MEG POA coordinates computed.")
                
                # Compute the POA distances per coordinates
                eeg_POA_dist = calculate_distances(mri_poa_coords, eeg_poa_coords, fmri_comparison_clusters)
                logger.info(f"        EEG-fMRI POA distances computed.")
                meg_POA_dist = calculate_distances(mri_poa_coords, meg_poa_coords, fmri_comparison_clusters)
                logger.info(f"        MEG-fMRI POA distances computed.")
                
                # Compute the COG of the different clusters
                try:
                    eeg_cog_coords = found_wcog(eeg_cluster_map[0], eeg_file, cog_index = [1])
                    logger.info(f"        EEG COG coordinates computed.")
                except Exception as e:
                    # Handle any exceptions that occur during overlap computation
                    logger.error(f"        {e}.")
                    eeg_cog_coords = np.array([[np.nan, np.nan, np.nan]])
                try:
                    meg_cog_coords = found_wcog(meg_cluster_map[0], meg_file, cog_index = [1])
                    logger.info(f"        MEG COG coordinates computed.")
                except Exception as e:
                    # Handle any exceptions that occur during overlap computation
                    logger.error(f"        {e}.")
                    meg_cog_coords = np.array([[np.nan, np.nan, np.nan]])
    
                # Compute the COG distances
                eeg_COG_dist = calculate_distances(mri_cogs_coords, eeg_cog_coords, fmri_comparison_clusters)
                logger.info(f"        EEG-fMRI COG distances computed.")
                meg_COG_dist = calculate_distances(mri_cogs_coords, meg_cog_coords, fmri_comparison_clusters)
                logger.info(f"        MEG-fMRI COG distances computed.")
    
                # Compute the overlap size between modalities
                try:
                    # Compute the overlap between the two closest clusters
                    eeg_jaccardidx = compute_overlap(mri_cluster_map[0], eeg_cluster_map[0], cluster_value = eeg_COG_dist['min_index'])
                    logger.info(f"        EEG-fMRI overlap computed.")
    
                except Exception as e:
                    # Handle any exceptions that occur during overlap computation
                    logger.error(f"        {e}")
                    eeg_jaccardidx = {'jaccard_index': np.nan, 'cluster_index': np.nan}
    
                try:
                    # Compute the overlap between the two closest clusters
                    meg_jaccardidx = compute_overlap(mri_cluster_map[0], meg_cluster_map[0], cluster_value = meg_COG_dist['min_index'])
                    logger.info(f"        MEG-fMRI overlap computed.")
                except Exception as e:
                    # Handle any exceptions that occur during overlap computation
                    logger.error(f"        {str(e)}")
                    meg_jaccardidx = {'jaccard_index': np.nan, 'cluster_index': np.nan}
                
               
                # Define the types of analysis and corresponding data
                analysis_types = ['POA', 'COG', 'Overlap']
                modal_data_list = [
                    (mri_poa_coords, eeg_poa_coords, meg_poa_coords),
                    (mri_cogs_coords, eeg_cog_coords, meg_cog_coords),
                    (mri_clusters_size, eeg_clusters_size, meg_clusters_size)
                ]
                comparison_data_list = [
                    (eeg_POA_dist, meg_POA_dist),
                    (eeg_COG_dist, meg_COG_dist),
                    (eeg_jaccardidx, meg_jaccardidx)
                ]

                # Iterate over the analysis types and their corresponding data
                for analysis_type, modal_data, comparison_data in zip(analysis_types, modal_data_list, comparison_data_list):   
                    df_sub_[analysis_type] = fill_sub_df(df_sub_[analysis_type], task, condi, tp, modal_data, comparison_data, analysis_type, fmri_comparison_clusters)
                    df_all_sub_[analysis_type] = fill_all_sub_df(df_all_sub_[analysis_type], sub, task, condi, tp, modal_data, comparison_data, analysis_type, fmri_comparison_clusters)

        # Save the dataframes after each task
        sub_outdir = sub_derivatives_outdir / 'results'
        general_outdir = sub_derivatives_outdir.parent / 'results'
        
        for analysis_type in analysis_types:
            # Save subject-specific DataFrame
            sub_outdir_type = sub_outdir / analysis_type 
            sub_outdir_type.mkdir(parents=True, exist_ok=True)
            sub_df_path = sub_outdir_type / f"{sub}_analysis-{analysis_type}_modality_comparison.csv"
            save_df_to_csv(df_sub_[analysis_type], sub_df_path)
            logger.info(f"      {analysis_type} {sub} dataframe saved at {sub_df_path}")
        
            # Save combined DataFrame for all subjects
            general_outdir_type = general_outdir / analysis_type
            general_outdir_type.mkdir(parents=True, exist_ok=True)
            all_sub_df_path = general_outdir_type / f"all_subjects_analysis-{analysis_type}_modality_comparison.csv"
            save_df_to_csv(df_all_sub_[analysis_type], all_sub_df_path)
            logger.info(f"      {analysis_type} All Subjects dataframe saved at {sub_df_path}")


display(df_all_sub_['POA'])

display(df_all_sub_['COG'])

display(df_all_sub_['Overlap'])
